# Imports

In [ ]:
from enum import StrEnum
import json
import os
from typing import Annotated

from huggingface_hub import AsyncInferenceClient
from huggingface_hub.inference._generated.types import ChatCompletionOutputToolCall
import lancedb
from openai import AsyncOpenAI, pydantic_function_tool
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
)
import pandas as pd
from pydantic import BaseModel, Field
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

from utils import create_tool_schema_for_function

load_dotenv()

True

# Retrieval Augmented Generation (RAG)


In [37]:
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)
db = lancedb.connect("./data/lance_db")
tbl = db.open_table("movies")

In [ ]:
# We create a BaseModel for the arguments to our function to easily create a JSON schema
class QueryMovieDB(BaseModel):
    text: str = Field(
        description="Query overviews of movies",
    )
    limit: int = Field(
        default=10,
        description="Number of results to return",
    )


def query_movie_db(
    text: str,
    limit: int = 5,
) -> pd.DataFrame:
    """
    Query the LanceDB movie database for movies with similar overviews to the input text.
    Args:
        text (str): The input text to query the database.
        limit (int): The number of results to return.
    Returns:
        pd.DataFrame: A DataFrame containing the results of the query.
    """
    q_emb = embedder.encode(text)
    # res = tbl.search(q_emb).limit(limit).select(["genres", "overview", "keywords", "original_title", "spoken_languages", ""]).to_pandas().drop(columns=["id", "vector", "_distance"])
    res = (
        tbl.search(q_emb).limit(limit).to_pandas().drop(columns=["vector", "_distance"])
    )
    return res


In [51]:
res = query_movie_db("air bud")
print(res.to_dict(orient="records"))

[{'budget': 3500000, 'genres': array(['Comedy'], dtype=object), 'keywords': array(['chicago', 'alcohol', 'cataclysm', 'guitar', 'medicine',
       'taxi driver', 'passenger', 'saxophone', 'stewardess', 'pilot',
       'airplane', 'fear of flying', 'air controller', 'landing',
       'autopilot', 'kiss', 'spoof', 'los angeles', 'alcohol abuse',
       'aftercreditsstinger', 'anarchic comedy'], dtype=object), 'original_language': 'en', 'overview': "Alcoholic pilot, Ted Striker has developed a fear of flying due to wartime trauma, but nevertheless boards a passenger jet in an attempt to woo back his stewardess girlfriend. Food poisoning decimates the passengers and crew, leaving it up to Striker to land the plane with the help of a glue-sniffing air traffic controller and Striker's vengeful former Air Force captain, who must both talk him down.", 'popularity': 46.116885, 'release_date': datetime.date(1980, 7, 2), 'revenue': 83453539, 'runtime': 88.0, 'spoken_languages': array(['English'],

In [6]:
pydantic_function_tool(QueryMovieDB)

{'type': 'function',
 'function': {'name': 'QueryMovieDB',
  'strict': True,
  'parameters': {'properties': {'text': {'description': 'Query overviews of movies',
     'title': 'Text',
     'type': 'string'},
    'limit': {'default': 10,
     'description': 'Number of results to return',
     'title': 'Limit',
     'type': 'integer'}},
   'required': ['text', 'limit'],
   'title': 'QueryMovieDB',
   'type': 'object',
   'additionalProperties': False}}}

In [ ]:
schema = create_tool_schema_for_function(query_movie_db, QueryMovieDB)
schema

{'type': 'function',
 'function': {'name': 'query_movie_db',
  'description': 'Query the LanceDB movie database for movies with similar overviews to the input text.',
  'parameters': {'properties': {'text': {'description': 'Query overviews of movies',
     'title': 'Text',
     'type': 'string'},
    'limit': {'default': 10,
     'description': 'Number of results to return',
     'title': 'Limit',
     'type': 'integer'}},
   'required': ['text'],
   'title': 'QueryMovieDB',
   'type': 'object'}}}

## Hugging Face InferenceClient

* [Hugging Face InferenceClient Function Calling](https://huggingface.co/docs/hugs/en/guides/function-calling)

In [43]:
hf_token = os.getenv("HF_TOKEN")
if hf_token is None:
    raise ValueError("HF_TOKEN environment variable not set")

hf_client = AsyncInferenceClient(
    provider="sambanova",
    api_key=hf_token,
)
# hf_client = AsyncOpenAI(
#     base_url="https://router.huggingface.co/sambanova",
#     api_key=hf_token,
# )

In [47]:
messages = [
    {
        "role": "system",
        "content": "Don't make assumptions about values. Ask for clarification if needed.",
    },
    {
        "role": "user",
        "content": "I'd like to watch a movie about a retired assassin who is forced back into the game.",
    },
]

response = await hf_client.chat_completion(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=messages,
    tools=[schema],
    tool_choice="auto",  # allow the model to choose to call tool, if any; others options: "required": call one or more tools
)
print(response.choices[0].message.tool_calls)

[ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments='{"limit":10,"text":"a retired assassin who is forced back into the game"}', name='query_movie_db', description=None), id='call_21ffa06783614af48d', type='function')]


## OpenAI

* [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling?api-mode=chat)
* [Generous free tier](https://platform.openai.com/docs/models/gpt-4.1-nano)

![](https://i.ibb.co/JwZtC9px/Screenshot-2025-05-20-235653.png "GPT-4.1-nano")

In [ ]:
oai_api_key = os.getenv("OPENAI_API_KEY")
if oai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set")
oai_client = AsyncOpenAI(api_key=oai_api_key)

In [ ]:
messages = [
    {
        "role": "system",
        "content": "Don't make assumptions about values. Ask for clarification if needed.",
    },
    {
        "role": "user",
        "content": "I'd like to watch a movie about a retired assassin who is forced back into the game.",
    },
]

response = await oai_client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    tools=[schema],
    tool_choice="auto",
)
print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_OzpdNcnpEV5AddmPbzF95FS9', function=Function(arguments='{"text":"retired assassin forced back into the game"}', name='query_movie_db'), type='function')]


In [ ]:
def call_function(name, args):
    if name == "query_movie_db":
        func = query_movie_db
    else:
        raise ValueError(f"Unknown function: {name}")
    try:
        # Call the function with the provided arguments
        result = func(**args)
    except Exception as e:
        # Handle any exceptions that occur during the function call
        print(f"Error calling function {name}: {e}")
        return None
    return result

def handle_tool_calls(tool_calls: list[ChatCompletionMessageToolCall] | list[ChatCompletionOutputToolCall], messages: list[dict]) -> list[dict]:
    if not tool_calls:
        return messages
    for tool_call in tool_calls:
        if isinstance(tool_call, ChatCompletionMessageToolCall):
            # Handle the tool call
            function_name = tool_call.function_call.name
            arguments = json.loads(tool_call.function_call.arguments)
            result = call_function(function_name, arguments)
            # Send the result back to the model
            response = oai_client.chat.completions.create(
                model="gpt-4.1-nano",
                messages=[
                    {
                        "role": "function",
                        "name": function_name,
                        "content": json.dumps(result),
                    }
                ],
            )
            print(response.choices[0].message.content)

# Check for function calls in LLM response

In [ ]:
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    func = tool_call.function
    func_name = func.name
    tool_call_id = tool_call.id
    func_args = json.loads(func.arguments)
    result = call_function(func_name, func_args)
    messages.append(
        {
            "role": "tool",
            "tool_call_id": tool_call_id,
            "content": json.dumps(result),
        }
    )

In [46]:
response.choices[0].message.tool_calls[0].function.arguments

'{"limit":10,"text":"a retired assassin who is forced back into the game"}'

# Sentiment Analysis

[Structured Output](https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat)

In [ ]:
class Polarity(StrEnum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"


class SentimentAnalysisOutput(BaseModel):
    polarity: Annotated[Polarity, "The sentiment polarity of the text"]
    confidence: Annotated[
        float,
        Field(
            description="The confidence score of the sentiment polarity between 0 and 1",
            ge=0.0,
            le=1.0,
        ),
    ]

In [ ]:
SentimentAnalysisOutput(polarity="positive", confidence=1.1)

ValidationError: 1 validation error for SentimentAnalysisOutput
confidence
  Input should be less than or equal to 1 [type=less_than_equal, input_value=1.1, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal